In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import miceforest as mf
import random
import sklearn.neighbors._base 
from xgbimputer import XGBImputer

In [32]:
df_ori = pd.read_csv("../datasets/train.csv")

In [33]:
# label drop
target = 'DC201'

df_ori = df_ori.drop('id', axis=1)
df_ori = df_ori.dropna(subset=target)

In [34]:
NUMERICAL_DATA = ['DC216', 'DC220', 'DC142a']
CATEGORICAL_DATA = [column for column in df_ori.columns if column != target and column not in NUMERICAL_DATA]

COLUMN_CATEGORICAL_INDEX = []

for column in CATEGORICAL_DATA:
    COLUMN_CATEGORICAL_INDEX.append(df_ori.columns.get_loc(column))

In [35]:
numerical_df = df_ori[NUMERICAL_DATA].copy()
categorical_df = df_ori[CATEGORICAL_DATA].astype('category').copy()
target_df = df_ori[target].replace({'Layak Minum':1, 'Tidak Layak Minum':0})

df_ori = pd.concat([numerical_df, categorical_df, target_df], axis=1)
df_ori

,DC216,DC220,DC142a,DC024,DC025,DC205,DC206,DC207,DC208,DC209,...,DC237e,DC237f,DC241,DC242,DC244,DC246,DC252,DC270a,DC109,DC201
0,1.0,80.0,60.0,11,2,16.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1,12.0,1
1,2.0,44.0,70.0,11,2,31.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2,31.0,1
2,2.0,35.0,75.0,11,2,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,4,11.0,1
3,3.0,79.0,85.0,11,2,12.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2.0,NaN,1.0,0.0,1.0,1,11.0,1
4,2.0,37.0,65.0,11,2,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2,11.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35968,1.0,21.0,9.0,94,1,17.0,1.0,0.0,0.0,0.0,...,0.0,0.0,3.0,NaN,0.0,0.0,0.0,1,21.0,1
35969,1.0,41.0,12.0,94,1,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2,11.0,1
35970,2.0,52.0,48.0,94,1,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1,11.0,1
35971,2.0,58.0,39.0,94,1,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2,11.0,1


In [36]:
df_ori.isna().sum()

DC216        39
DC220         4
DC142a       52
DC024         0
DC025         0
DC205        10
DC206        21
DC207        43
DC208        13
DC209        18
DC210        41
DC211        24
DC212        24
DC213        77
DC214         9
DC215        14
DC217         0
DC219         0
DC226         6
DC230a       11
DC230b     3009
DC232      3024
DC232b     3024
DC235     10444
DC237        19
DC237a       35
DC237b       35
DC237c       35
DC237d       35
DC237e       35
DC237f       35
DC241       612
DC242      5374
DC244         4
DC246         9
DC252        14
DC270a        0
DC109        10
DC201         0
dtype: int64

In [37]:
# Create kernels.  # Mice Forest
kernel = mf.ImputationKernel(
  data=df_ori,
  save_all_iterations=True,
  categorical_feature='auto',
  random_state=1343
)

kernel.mice(5,verbose=True)

completed_dataset_mf = kernel.complete_data(dataset=0, inplace=False)

/home/codespace/.python/current/lib/python3.10/site-packages/miceforest/ImputationKernel.py:371: UserWarning: [DC205,DC213,DC214,DC215,DC217,DC226,DC237,DC237a,DC237b,DC237c,DC237d,DC237e,DC237f,DC241,DC109] have very rare categories, it is a good idea to group these, or set the min_data_in_leaf parameter to prevent lightgbm from outputting 0.0 probabilities.
  warn(


Initialized logger with name mice 1-5
Dataset 0
1  | DC220 | DC244 | DC226 | DC246 | DC214 | DC205 | DC109 | DC230a | DC208 | DC215 | DC252 | DC209 | DC237 | DC206 | DC212 | DC211 | DC237f | DC237e | DC237d | DC237c | DC237b | DC237a | DC216 | DC210 | DC207 | DC142a | DC213 | DC241 | DC230b | DC232b | DC232 | DC242 | DC235
2  | DC220 | DC244 | DC226 | DC246 | DC214 | DC205 | DC109 | DC230a | DC208 | DC215 | DC252 | DC209 | DC237 | DC206 | DC212 | DC211 | DC237f | DC237e | DC237d | DC237c | DC237b | DC237a | DC216 | DC210 | DC207 | DC142a | DC213 | DC241 | DC230b | DC232b | DC232 | DC242 | DC235
3  | DC220 | DC244 | DC226 | DC246 | DC214 | DC205 | DC109 | DC230a | DC208 | DC215 | DC252 | DC209 | DC237 | DC206 | DC212 | DC211 | DC237f | DC237e | DC237d | DC237c | DC237b | DC237a | DC216 | DC210 | DC207 | DC142a | DC213 | DC241 | DC230b | DC232b | DC232 | DC242 | DC235
4  | DC220 | DC244 | DC226 | DC246 | DC214 | DC205 | DC109 | DC230a | DC208 | DC215 | DC252 | DC209 | DC237 | DC206 | DC2

In [38]:
completed_dataset_mf

,DC216,DC220,DC142a,DC024,DC025,DC205,DC206,DC207,DC208,DC209,...,DC237e,DC237f,DC241,DC242,DC244,DC246,DC252,DC270a,DC109,DC201
0,1.0,80.0,60.0,11,2,16.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1,12.0,1
1,2.0,44.0,70.0,11,2,31.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2,31.0,1
2,2.0,35.0,75.0,11,2,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,4,11.0,1
3,3.0,79.0,85.0,11,2,12.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,1.0,0.0,1.0,1,11.0,1
4,2.0,37.0,65.0,11,2,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2,11.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35968,1.0,21.0,9.0,94,1,17.0,1.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1,21.0,1
35969,1.0,41.0,12.0,94,1,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2,11.0,1
35970,2.0,52.0,48.0,94,1,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1,11.0,1
35971,2.0,58.0,39.0,94,1,12.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2,11.0,1


In [39]:
completed_dataset_mf.nunique()

DC216      15
DC220      84
DC142a    362
DC024      34
DC025       2
DC205       6
DC206       2
DC207       2
DC208       2
DC209       2
DC210       2
DC211       2
DC212       2
DC213      10
DC214      15
DC215      12
DC217       6
DC219       2
DC226      11
DC230a      5
DC230b      2
DC232       2
DC232b      2
DC235       3
DC237       3
DC237a      3
DC237b      3
DC237c      3
DC237d      3
DC237e      3
DC237f      3
DC241       4
DC242       2
DC244       2
DC246       2
DC252       5
DC270a      5
DC109       9
DC201       2
dtype: int64

In [40]:
completed_dataset_mf.to_csv('no_map_imputed_mf5iter.csv', index=False)